In [3]:
import datajoint as dj

In [4]:
import datajoint as dj
import numpy as np
m65 = dj.create_virtual_module('m65', 'microns_minnie65_01')
schema = dj.schema("microns_minnie65_01")
dj.config["display.limit"] = 20

Connecting celiib@10.28.0.34:3306


In [ ]:
@schema
class NullModelParameters(dj.Manual):
    definition="""
    n
    """

In [ ]:
decimation_version = 0
decimation_ratio = 0.25
import time


@schema
class MultiSomaCentroidValidation(dj.Computed):
    definition="""
    -> minnie.Decimation
    soma_index : tinyint unsigned #index given to this soma to account for multiple somas in one base semgnet
    ---
    centroid_x           : int unsigned                 # (EM voxels)
    centroid_y           : int unsigned                 # (EM voxels)
    centroid_z           : int unsigned                 # (EM voxels)
    distance_from_prediction : double                   # the distance of the ALLEN predicted centroid soma center from the algorithms prediction
    prediction_matching_index : int unsigned            # the soma index that was used to compute the error
    soma_vertices             : longblob                # array of vertices
    soma_faces             : longblob                   # array of faces
    multiplicity         : tinyint unsigned             # the number of somas found for this base segment
    run_time : double                   # the amount of time to run (seconds)

    """
    
    #key_source = (minnie.Decimation & (dj.U("segment_id") & (m65.AllenSegmentCentroid & "status=1").proj()) & "version=" + str(decimation_version))
    key_source = minnie.Decimation() & (m65.AllenMultiSomas() & "status > 0").proj()
    
    def make(self,key):
        #get the mesh data
        print(f"\n\n\n---- Working on {key['segment_id']} ----")
        
        new_mesh = (minnie.Decimation() & key).fetch1("mesh")
        current_mesh_verts,current_mesh_faces = new_mesh.vertices,new_mesh.faces
        
        segment_id = key["segment_id"]
        
        total_soma_list, run_time = extract_soma_center(segment_id,current_mesh_verts,current_mesh_faces)
        print(f"Run time was {run_time}  and the total_soma_list = {total_soma_list} and ")
        
        #check if soma list is empty and did not find soma
        if len(total_soma_list) <= 0:
            print("There were no somas found for this mesh so just writing empty data")
            insert_dict = dict(key,
                              soma_index=-1,
                              centroid_x=None,
                               centroid_y=None,
                               centroid_z=None,
                               distance_from_prediction=None,
                               prediction_matching_index = None,
                               soma_vertices=None,
                               soma_faces=None,
                               multiplicity=0,
                               run_time=run_time
                              )
            
            raise Exception("to prevent writing because none were found")
            self.insert1(insert_dict,skip_duplicates=True)
            return
        
        #if there is one or more soma found
        
        #get the soma prediction
        
        
        new_array = (m65.AllenMultiSomas.Centroids()  & key).fetch("centroid_x","centroid_y","centroid_z")
        soma_ids = (m65.AllenMultiSomas.Centroids()  & key).fetch("soma_id")
        allen_centroid_prediction = np.array(new_array).T
        allen_centroid_prediction
        #print("soma_ids = " + str(soma_ids))
        from pykdtree.kdtree import KDTree
        mesh_tree = KDTree(allen_centroid_prediction)

        
        dicts_to_insert = []
        

            
        for i,current_soma in enumerate(total_soma_list):
            print("Trying to write off file")
            current_soma.export(f"{key['segment_id']}/{key['segment_id']}_soma_{i}.off")
            auto_prediction_center = np.mean(current_soma.vertices,axis=0)
            
            distances,closest_node = mesh_tree.query(auto_prediction_center.reshape(1,3))
            error_distance = distances[0]
            prediction_matching_index = soma_ids[closest_node[0]] #closest nodes and the distances
            
            
            insert_dict = dict(key,
                              soma_index=i,
                              centroid_x=auto_prediction_center[0],
                               centroid_y=auto_prediction_center[1],
                               centroid_z=auto_prediction_center[2],
                               distance_from_prediction=error_distance,
                               prediction_matching_index = prediction_matching_index,
                               soma_vertices=current_soma.vertices,
                               soma_faces=current_soma.faces,
                               multiplicity=len(total_soma_list),
                               run_time=run_time
                              )
            
            
            
            dicts_to_insert.append(insert_dict)
            
        #raise Exception("to prevent writing")
        
        if len(total_soma_list) != len(soma_ids):
            raise Exception("to prevent writing SOMAS NOT EQUAL TO That registered in datase")
            
        self.insert(dicts_to_insert,skip_duplicates=True)